In [ ]:
!pip install jiwer -q
!pip install torchaudio -q
!pip install git+https://github.com/huggingface/transformers.git -q
!pip install librosa==0.8.1 -q
!pip install git+https://github.com/huggingface/datasets.git -q
!pip install transformers==4.21.1 -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 33.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 58.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 81.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 35.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.8/203.8 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 57.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.6 MB/s eta 0:00:00
  

In [ ]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchaudio
from transformers import AutoConfig, Wav2Vec2Processor

import librosa
import IPython.display as ipd
import numpy as np
import pandas as pd

In [ ]:
# load model
!mkdir -p /content/data
file_id = "1CeUHCmCKCnjlibvP6MPn4WqyOFAaMrfv"
!gdown --id $file_id -O /content/data/model_1.zip
!unzip -q /content/data/model_1.zip -d /content/data/

/usr/local/lib/python3.10/dist-packages/gdown/cli.py:121: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1CeUHCmCKCnjlibvP6MPn4WqyOFAaMrfv
To: /content/data/model_1.zip
100% 1.17G/1.17G [00:17<00:00, 66.8MB/s]


In [ ]:
!mkdir -p /content/test_data
file_id = "1C-XEnwmfvxGjDDvrsI4M8jXuBPE_AOr_"
!gdown --id $file_id -O /content/test_data/test_emotion_dataset.zip
!unzip -q /content/test_data/test_emotion_dataset.zip -d /content/test_data/

/usr/local/lib/python3.10/dist-packages/gdown/cli.py:121: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1C-XEnwmfvxGjDDvrsI4M8jXuBPE_AOr_
To: /content/test_data/test_emotion_dataset.zip
100% 124M/124M [00:00<00:00, 140MB/s]


In [ ]:
folder_path = '/content/test_data/original'

# Списки для хранения путей и имен файлов
file_paths = []
file_names = []

# Обойдем файлы в папке
for root, dirs, files in os.walk(folder_path):
    for file in files:
        file_paths.append(os.path.join(root, file))
        file_names.append(file)

# Создаем DataFrame
data = {'File Path': file_paths, 'File Name': file_names}
test_data = pd.DataFrame(data)

# Вывести первые строки DataFrame
test_data

,File Path,File Name
0,/content/test_data/original/8_26_1.wav,8_26_1.wav
1,/content/test_data/original/10_42_0.wav,10_42_0.wav
2,/content/test_data/original/4_40_1.wav,4_40_1.wav
3,/content/test_data/original/1_43_2.wav,1_43_2.wav
4,/content/test_data/original/10_34_1.wav,10_34_1.wav
...,...,...
1545,/content/test_data/original/6_34_4.wav,6_34_4.wav
1546,/content/test_data/original/5_39_1.wav,5_39_1.wav
1547,/content/test_data/original/2_26_1.wav,2_26_1.wav
1548,/content/test_data/original/4_21_4.wav,4_21_4.wav


In [ ]:
from dataclasses import dataclass
from typing import Optional, Tuple
import torch
from transformers.file_utils import ModelOutput


@dataclass
class SpeechClassifierOutput(ModelOutput):
    loss: Optional[torch.FloatTensor] = None
    logits: torch.FloatTensor = None
    hidden_states: Optional[Tuple[torch.FloatTensor]] = None
    attentions: Optional[Tuple[torch.FloatTensor]] = None

In [ ]:
import torch
import torch.nn as nn
from torch.nn import BCEWithLogitsLoss, CrossEntropyLoss, MSELoss

from transformers.models.wav2vec2.modeling_wav2vec2 import (
    Wav2Vec2PreTrainedModel,
    Wav2Vec2Model
)


class Wav2Vec2ClassificationHead(nn.Module):
    """Head for wav2vec classification task."""

    def __init__(self, config):
        super().__init__()
        self.dense = nn.Linear(config.hidden_size, config.hidden_size)
        self.dropout = nn.Dropout(config.final_dropout)
        self.out_proj = nn.Linear(config.hidden_size, config.num_labels)

    def forward(self, features, **kwargs):
        x = features
        x = self.dropout(x)
        x = self.dense(x)
        x = torch.tanh(x)
        x = self.dropout(x)
        x = self.out_proj(x)
        return x


class Wav2Vec2ForSpeechClassification(Wav2Vec2PreTrainedModel):
    def __init__(self, config):
        super().__init__(config)
        self.num_labels = config.num_labels
        self.pooling_mode = config.pooling_mode
        self.config = config

        self.wav2vec2 = Wav2Vec2Model(config)
        self.classifier = Wav2Vec2ClassificationHead(config)

        self.init_weights()

    def freeze_feature_extractor(self):
        self.wav2vec2.feature_extractor._freeze_parameters()

    def merged_strategy(
            self,
            hidden_states,
            mode="mean"
    ):
        if mode == "mean":
            outputs = torch.mean(hidden_states, dim=1)
        elif mode == "sum":
            outputs = torch.sum(hidden_states, dim=1)
        elif mode == "max":
            outputs = torch.max(hidden_states, dim=1)[0]
        else:
            raise Exception(
                "The pooling method hasn't been defined! Your pooling mode must be one of these ['mean', 'sum', 'max']")

        return outputs

    def forward(
            self,
            input_values,
            attention_mask=None,
            output_attentions=None,
            output_hidden_states=None,
            return_dict=None,
            labels=None,
    ):
        return_dict = return_dict if return_dict is not None else self.config.use_return_dict
        outputs = self.wav2vec2(
            input_values,
            attention_mask=attention_mask,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
        )
        hidden_states = outputs[0]
        hidden_states = self.merged_strategy(hidden_states, mode=self.pooling_mode)
        logits = self.classifier(hidden_states)

        loss = None
        if labels is not None:
            if self.config.problem_type is None:
                if self.num_labels == 1:
                    self.config.problem_type = "regression"
                elif self.num_labels > 1 and (labels.dtype == torch.long or labels.dtype == torch.int):
                    self.config.problem_type = "single_label_classification"
                else:
                    self.config.problem_type = "multi_label_classification"

            if self.config.problem_type == "regression":
                loss_fct = MSELoss()
                loss = loss_fct(logits.view(-1, self.num_labels), labels)
            elif self.config.problem_type == "single_label_classification":
                loss_fct = CrossEntropyLoss()
                loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))
            elif self.config.problem_type == "multi_label_classification":
                loss_fct = BCEWithLogitsLoss()
                loss = loss_fct(logits, labels)

        if not return_dict:
            output = (logits,) + outputs[2:]
            return ((loss,) + output) if loss is not None else output

        return SpeechClassifierOutput(
            loss=loss,
            logits=logits,
            hidden_states=outputs.hidden_states,
            attentions=outputs.attentions,
        )


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_name_or_path = "/content/data"
config = AutoConfig.from_pretrained(model_name_or_path)
processor = Wav2Vec2Processor.from_pretrained(model_name_or_path)
sampling_rate = processor.feature_extractor.sampling_rate
model = Wav2Vec2ForSpeechClassification.from_pretrained(model_name_or_path).to(device)

In [ ]:
def speech_file_to_array_fn(path, sampling_rate=16000):
    speech_array, _sampling_rate = torchaudio.load(path)
    resampler = torchaudio.transforms.Resample(_sampling_rate)
    speech = resampler(speech_array).squeeze().numpy()
    return speech


def predict(path, sampling_rate=16000):
    speech = speech_file_to_array_fn(path, sampling_rate)
    features = processor(speech, sampling_rate=sampling_rate, return_tensors="pt", padding=True)

    input_values = features.input_values.to(device)
    attention_mask = features.attention_mask.to(device)

    with torch.no_grad():
        logits = model(input_values, attention_mask=attention_mask).logits

    scores = F.softmax(logits, dim=1).detach().cpu().numpy()[0]
    outputs = [{"Emotion": config.id2label[i], "Score": f"{round(score * 100, 3):.1f}%"} for i, score in enumerate(scores)]
    return outputs

In [ ]:
predict('/content/test_data/original/10_20_4.wav')

[{'Emotion': 'angry', 'Score': '7.3%'},
 {'Emotion': 'calm', 'Score': '0.8%'},
 {'Emotion': 'disgust', 'Score': '76.6%'},
 {'Emotion': 'fearful', 'Score': '0.3%'},
 {'Emotion': 'happy', 'Score': '0.6%'},
 {'Emotion': 'neutral', 'Score': '0.2%'},
 {'Emotion': 'sad', 'Score': '0.7%'},
 {'Emotion': 'surprised', 'Score': '13.5%'}]

In [ ]:
# Функция для записи результатов в текстовый файл
def write_results_to_file(file_id, emotions, file_name, output_file):
    emotion_values = [emotion['Score'] for emotion in emotions]
    with open(output_file, 'a') as f:
        f.write(f'file_id  |01   | 02  | 03  | 04  | 05  | 06  | 07  | 08  |\n')
        f.write(f'{file_name}|' + '|'.join(emotion_values) + '|\n')

# Путь к папке с исходными аудиофайлами
input_dir = '/content/test_data/original'

# Создайте папку для результатов, если она не существует
output_dir = '/content/results'
os.makedirs(output_dir, exist_ok=True)

# Получите список файлов в папке
file_list = os.listdir(input_dir)

# Пройдите по каждому файлу и примените функцию predict
for file in file_list:
    if file.endswith('.wav'):
        file_path = os.path.join(input_dir, file)
        emotions = predict(file_path)
        file_id = file.split('.')[0]
        write_results_to_file(file_id, emotions, file, os.path.join(output_dir, f'{file_id}_results.txt'))

In [ ]:
import os
from google.colab import files

# Путь к папке с результатами
output_dir = '/content/results'

# Создайте архив с результатами
os.system(f'zip -r /content/results.zip {output_dir}')

# Скачайте архив на локальный компьютер
files.download('/content/results.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>